created by Ignacio Oguiza - email: oguiza@gmail.com

# How to efficiently work with very large Numpy Arrays?

## Import libraries

In [1]:
import sys
import os
ISCOLAB = 'google.colab' in sys.modules
if ISCOLAB:
    if not os.path.isdir('/content/timeseriesAI2'):
        !git clone https://github.com/timeseriesAI/timeseriesAI2.git
        !pip install git+https://github.com/fastai/fastai2 
        !pip install git+https://github.com/fastai/fastcore 
        !pip install pyunpack
        !pip install sktime
        %cd timeseriesAI2
    else: 
        path = !pwd
        if path != ['/content/timeseriesAI2']: 
            %cd timeseriesAI2
        !pwd

## Introduction

I normally work with time series data. I made the decision to use numpy arrays to store my data since the can easily handle multiple dimensions, and are really very efficient.

But sometimes datasets are really big (many GBs) and don't fit in memory. So I started looking around and found something that works very well: [**np.memmap**](https://docs.scipy.org/doc/numpy/reference/generated/numpy.memmap.html). Conceptually they work as arrays on disk, and that's how I often call them.

np.memmap creates a map to numpy array you have previously saved on disk, so that you can efficiently access small segments of those (small or large) files on disk, without reading the entire file into memory. And that's exactly what we need with deep learning, be able to quickly create a batch in memory, without reading the entire file (that is stored on disk). 

The best analogy I've found are image files. You may have a very large dataset on disk (that far exceeds your RAM). In order to create your DL datasets, what you pass are the paths to each individual file, so that you can then load a few images and create a batch on demand.

You can view np.memmap as the path collection that can be used to load numpy data on demand when you need to create a batch.

So let's see how you can work with larger than RAM arrays on disk.

On my laptop I have only 8GB of RAM.

In [2]:
from fastai2.torch_core import *
from timeseries.imports import *
from timeseries.utils import *
print(f'Total RAM      : {bytes2GB(psutil.virtual_memory().total):5.2f} GB')
print(f'Available RAM  : {bytes2GB(psutil.virtual_memory().available):5.2f} GB\n')

Total RAM      :  8.00 GB
Available RAM  :  3.75 GB



I will try to demonstrate how you can handle a 10 GB numpy array dataset in an efficient way. 

## Create and save a larger than memory array

I will now create a large numpy array that doesn't fit in memory. 
Since I don't have enough RAM, I'll create an empty array on disk, and then load data in chunks that fit in memory.

⚠️ If you want to to experiment with large datasets, you may uncomment and run this code. **It will create a ~10GB on your disk**. 
If you do it, remember to delete it later.
In my laptop it took me around **11 mins to run.**

In [3]:
# # Save a small empty array
# X_temp_fn = './data/temp_X.npy'
# np.save(X_temp_fn, np.empty(1))

# # Create a np.memmap with desired dtypes and shape of the large array you want to save.
# # It's just a placeholder that doesn't contain any data
# X_fn = './data/X_on_disk.npy'
# X = np.memmap(X_temp_fn, dtype='float32', shape=(100000, 50, 512))

# # We are going to create a loop to fill in the np.memmap
# start = 0
# for i in range(20):
#     # You now grab a chunk of your data that fits in memory
#     # This could come from a pandas dataframe for example
#     # I will simulate it with some random data
#     data_chunk = np.random.rand(5000, 50, 512)
#     end = start + data_chunk.shape[0]
    
#     # I now fill a slice of the np.memmap
#     X[start:end] = data_chunk
    
#     start = end
#     del data_chunk

# #I can now remove the temp file I created
# os.remove(X_temp_fn)

# # Once the data is loaded on the np.memmap, I save it as a normal np.array
# np.save(X_fn, X)

# # I will create a smaller array. Sinc this fits in memory, I don't need to use a memmap
# y_fn = 'y_on_disk.npy'
# y = np.random.randint(0, 10, X.shape[0])
# labels = np.array(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j'])
# np.save(y_fn, labels[y])

# del X, y

Ok. So let's check the size of these files on memory.

In [4]:
print(f'X array: {os.path.getsize("./data/X_on_disk.npy"):12} bytes ({bytes2GB(os.path.getsize("./data/X_on_disk.npy")):3.3f} GB)')
print(f'y array: {os.path.getsize("./data/y_on_disk.npy"):12} bytes ({bytes2GB(os.path.getsize("./data/y_on_disk.npy")):3.3f} GB)')

X array:  10240000128 bytes (9.540 GB)
y array:       400128 bytes (0.000 GB)


## Load an array on disk (np.memmap)

Remember I only have an 8 GB RAM on this laptop, so I couldn't load these datasets in memory.

☣️ Actually I accidentally loaded the "X_on_disk.npy" file, and my laptop crahsed so I had to reboot it!

So let's now load data as arrays on disk (np.memmap). The way to do it is super simple, and very efficient. You just do it as you would with a normal array, but add an mmap_mode.

There are 4 modes: 

- ‘r’	Open existing file for reading only.
- ‘r+’	Open existing file for reading and writing.
- ‘w+’	Create or overwrite existing file for reading and writing.
- ‘c’	Copy-on-write: assignments affect data in memory, but changes are not saved to disk. The file on disk is read-only.

I normally use mode 'r' since I want to be able to make changes to data in memory (transforms for example), without affecting data on disk (same approach as with image data). This is the same thing you do with image files on disk, that are just read, and then modified in memory, without change the file on disk.

In [38]:
X_on_disk = np.load('./data/X_on_disk.npy', mmap_mode='r')
y_on_disk = np.load('./data/y_on_disk.npy', mmap_mode='r')

(100000, 50, 512)


**Fast load**: it only takes a few ms to "load" a memory map to a 10 GB array on disk.

In fact, the only thing that is loaded is a map to the array stored on disk. That's why it's so fast.

## Arrays on disk: main features

### Very limited RAM usage

In [39]:
print(X_on_disk.shape, y_on_disk.shape)

(100000, 50, 512) (100000,)


In [6]:
print(f'X array on disk: {sys.getsizeof(X_on_disk):12} bytes ({bytes2GB(sys.getsizeof(X_on_disk)):3.3f} GB)')
print(f'y array on disk: {sys.getsizeof(y_on_disk):12} bytes ({bytes2GB(sys.getsizeof(y_on_disk)):3.3f} GB)')

X array on disk:          152 bytes (0.000 GB)
y array on disk:          120 bytes (0.000 GB)


**152 bytes of RAM for a 10GB array**. This is the great benefit of arrays on disk.

Arrays on disk barely use any RAM until each the it's sliced and an element is converted into a np.array or a tensor.

This is equivalent to the size of file paths in images (very limited) compared to the files themselves (actual images). 

### Types

np.memmap is a subclass of np.ndarray

In [7]:
isinstance(X_on_disk, np.ndarray)

True

In [8]:
type(X_on_disk)

numpy.memmap

### Operations

With np.memmap you can perform the same operations you would with a normal numpy array. 
The most common operations you will perform in deep learning are:

- slicing
- calculating stats: mean and std
- scaling (using normalize or standardize)
- transformation into a tensor

Once you get the array on disk slice, you'll convert it into a tensor, move to a GPU and performs operations there.


⚠️ You need to be careful though not to convert the entire np.memmap to an array/ tensor if it's larger than your RAM. This will crash your computer unless you have enough RAM, so you would have to reboot!

**DON'T DO THIS:  torch.from_numpy(X) or np.array(X)** unless you have ehough RAM.

To avoid issues during test, I created a smaller array on disk (that I can store in memory). When I want to test something I test it with that array first. It's important to always verify that the type output of your operations is np.memmap, which means data is still in memory.

#### Slicing

To ensure you don't brind the entire array in memory (which may crash your computer) you can always work with slices of data, which is by the way how fastai works.

If you use mode 'r' you can grab a sample and make changes to it, but this won't modify data on disk.

In [9]:
x = X_on_disk[0]
x

memmap([[0.2626543 , 0.81090355, 0.32464218, ..., 0.23839498, 0.67757344,
         0.3310132 ],
        [0.7845197 , 0.68123066, 0.38421404, ..., 0.5383686 , 0.01147997,
         0.13235402],
        [0.6679299 , 0.56707335, 0.02308166, ..., 0.77351165, 0.20462573,
         0.9898304 ],
        ...,
        [0.38630033, 0.5770135 , 0.18095827, ..., 0.9798217 , 0.56657016,
         0.6018766 ],
        [0.09030128, 0.01747155, 0.5882474 , ..., 0.28344667, 0.6954193 ,
         0.93599343],
        [0.91863537, 0.01252496, 0.4146458 , ..., 0.80348647, 0.9133351 ,
         0.5418618 ]], dtype=float32)

It's important to note thatn when we perform an math operation on a np.memmap (add, subtract, ...) the output is a np.array, and no longer a np.memmap. 

⚠️ Remember you don't want to run this type of operations with a memmap larger than your RAM!! That's why I do it with a slice.

In [10]:
x = X_on_disk[0] + 1
x

array([[1.2626543, 1.8109035, 1.3246422, ..., 1.238395 , 1.6775734,
        1.3310132],
       [1.7845197, 1.6812307, 1.384214 , ..., 1.5383686, 1.01148  ,
        1.132354 ],
       [1.6679299, 1.5670733, 1.0230817, ..., 1.7735116, 1.2046257,
        1.9898304],
       ...,
       [1.3863003, 1.5770135, 1.1809583, ..., 1.9798217, 1.5665702,
        1.6018766],
       [1.0903013, 1.0174716, 1.5882474, ..., 1.2834467, 1.6954193,
        1.9359934],
       [1.9186354, 1.012525 , 1.4146458, ..., 1.8034865, 1.9133351,
        1.5418618]], dtype=float32)

In [11]:
x = torch.from_numpy(X_on_disk[0])
x2 = x + 1
x2

tensor([[1.2627, 1.8109, 1.3246,  ..., 1.2384, 1.6776, 1.3310],
        [1.7845, 1.6812, 1.3842,  ..., 1.5384, 1.0115, 1.1324],
        [1.6679, 1.5671, 1.0231,  ..., 1.7735, 1.2046, 1.9898],
        ...,
        [1.3863, 1.5770, 1.1810,  ..., 1.9798, 1.5666, 1.6019],
        [1.0903, 1.0175, 1.5882,  ..., 1.2834, 1.6954, 1.9360],
        [1.9186, 1.0125, 1.4146,  ..., 1.8035, 1.9133, 1.5419]])

As you can see, this doesn't affect the original np.memmap

In [12]:
X_on_disk[0]

memmap([[0.2626543 , 0.81090355, 0.32464218, ..., 0.23839498, 0.67757344,
         0.3310132 ],
        [0.7845197 , 0.68123066, 0.38421404, ..., 0.5383686 , 0.01147997,
         0.13235402],
        [0.6679299 , 0.56707335, 0.02308166, ..., 0.77351165, 0.20462573,
         0.9898304 ],
        ...,
        [0.38630033, 0.5770135 , 0.18095827, ..., 0.9798217 , 0.56657016,
         0.6018766 ],
        [0.09030128, 0.01747155, 0.5882474 , ..., 0.28344667, 0.6954193 ,
         0.93599343],
        [0.91863537, 0.01252496, 0.4146458 , ..., 0.80348647, 0.9133351 ,
         0.5418618 ]], dtype=float32)

You can slice an array on disk by any axis, and it'll return a memmap. Slicing by any axis is very fast.

In [13]:
X_on_disk[0]

memmap([[0.2626543 , 0.81090355, 0.32464218, ..., 0.23839498, 0.67757344,
         0.3310132 ],
        [0.7845197 , 0.68123066, 0.38421404, ..., 0.5383686 , 0.01147997,
         0.13235402],
        [0.6679299 , 0.56707335, 0.02308166, ..., 0.77351165, 0.20462573,
         0.9898304 ],
        ...,
        [0.38630033, 0.5770135 , 0.18095827, ..., 0.9798217 , 0.56657016,
         0.6018766 ],
        [0.09030128, 0.01747155, 0.5882474 , ..., 0.28344667, 0.6954193 ,
         0.93599343],
        [0.91863537, 0.01252496, 0.4146458 , ..., 0.80348647, 0.9133351 ,
         0.5418618 ]], dtype=float32)

In [14]:
X_on_disk[:, 0]

memmap([[0.2626543 , 0.81090355, 0.32464218, ..., 0.23839498, 0.67757344,
         0.3310132 ],
        [0.7720094 , 0.21011464, 0.02604653, ..., 0.8257726 , 0.9297855 ,
         0.34065437],
        [0.10447659, 0.7670673 , 0.838875  , ..., 0.76260966, 0.5328985 ,
         0.2714968 ],
        ...,
        [0.8387722 , 0.13451461, 0.8197776 , ..., 0.3349404 , 0.43819886,
         0.65123564],
        [0.20458107, 0.76076484, 0.5841517 , ..., 0.8807168 , 0.8641069 ,
         0.5569748 ],
        [0.6686797 , 0.9680852 , 0.04992276, ..., 0.99571806, 0.22515585,
         0.9119718 ]], dtype=float32)

However, bear in mind that if you use multiple indices, the output will be a regular numpy array. This is important as it will use more RAM. 

In [15]:
X_on_disk[[0,1]]

array([[[0.2626543 , 0.81090355, 0.32464218, ..., 0.23839498,
         0.67757344, 0.3310132 ],
        [0.7845197 , 0.68123066, 0.38421404, ..., 0.5383686 ,
         0.01147997, 0.13235402],
        [0.6679299 , 0.56707335, 0.02308166, ..., 0.77351165,
         0.20462573, 0.9898304 ],
        ...,
        [0.38630033, 0.5770135 , 0.18095827, ..., 0.9798217 ,
         0.56657016, 0.6018766 ],
        [0.09030128, 0.01747155, 0.5882474 , ..., 0.28344667,
         0.6954193 , 0.93599343],
        [0.91863537, 0.01252496, 0.4146458 , ..., 0.80348647,
         0.9133351 , 0.5418618 ]],

       [[0.7720094 , 0.21011464, 0.02604653, ..., 0.8257726 ,
         0.9297855 , 0.34065437],
        [0.7020411 , 0.7310075 , 0.00217595, ..., 0.74274397,
         0.85443044, 0.469761  ],
        [0.7890441 , 0.29349124, 0.4332237 , ..., 0.22531688,
         0.13641207, 0.955273  ],
        ...,
        [0.08417109, 0.47016835, 0.9377146 , ..., 0.82218635,
         0.58684355, 0.44834593],
        [0.1

There's a trick we can use avoid this making use of the excellent new L class in fastai. It is to **itemify** the np.memmap/s. 

In [40]:
def itemify(*x): return L(*x).zip()

To itemify one or several np.memmap/s is very fast. Let's see how long it takes with a 10 GB array.

In [41]:
X_on_disk_as_items = itemify(X_on_disk)

5 seconds to return individual records on disk! Bear in mind you only need to perform this once!

So now, you can select multiple items at the same time, and they will all still be on disk:

In [18]:
X_on_disk_as_items[0,1]

(#2) [(memmap([[0.2626543 , 0.81090355, 0.32464218, ..., 0.23839498, 0.67757344,
         0.3310132 ],
        [0.7845197 , 0.68123066, 0.38421404, ..., 0.5383686 , 0.01147997,
         0.13235402],
        [0.6679299 , 0.56707335, 0.02308166, ..., 0.77351165, 0.20462573,
         0.9898304 ],
        ...,
        [0.38630033, 0.5770135 , 0.18095827, ..., 0.9798217 , 0.56657016,
         0.6018766 ],
        [0.09030128, 0.01747155, 0.5882474 , ..., 0.28344667, 0.6954193 ,
         0.93599343],
        [0.91863537, 0.01252496, 0.4146458 , ..., 0.80348647, 0.9133351 ,
         0.5418618 ]], dtype=float32),),(memmap([[0.7720094 , 0.21011464, 0.02604653, ..., 0.8257726 , 0.9297855 ,
         0.34065437],
        [0.7020411 , 0.7310075 , 0.00217595, ..., 0.74274397, 0.85443044,
         0.469761  ],
        [0.7890441 , 0.29349124, 0.4332237 , ..., 0.22531688, 0.13641207,
         0.955273  ],
        ...,
        [0.08417109, 0.47016835, 0.9377146 , ..., 0.82218635, 0.58684355,
         0

You can also itemify several items at once: X and y for example. When you slice the list, you'll get tuples.

In [42]:
Xy_on_disk_as_items = itemify(X_on_disk, y_on_disk)

In [43]:
Xy_on_disk_as_items[0, 1]

(#2) [(memmap([[0.2626543 , 0.81090355, 0.32464218, ..., 0.23839498, 0.67757344,
         0.3310132 ],
        [0.7845197 , 0.68123066, 0.38421404, ..., 0.5383686 , 0.01147997,
         0.13235402],
        [0.6679299 , 0.56707335, 0.02308166, ..., 0.77351165, 0.20462573,
         0.9898304 ],
        ...,
        [0.38630033, 0.5770135 , 0.18095827, ..., 0.9798217 , 0.56657016,
         0.6018766 ],
        [0.09030128, 0.01747155, 0.5882474 , ..., 0.28344667, 0.6954193 ,
         0.93599343],
        [0.91863537, 0.01252496, 0.4146458 , ..., 0.80348647, 0.9133351 ,
         0.5418618 ]], dtype=float32), 'b'),(memmap([[0.7720094 , 0.21011464, 0.02604653, ..., 0.8257726 , 0.9297855 ,
         0.34065437],
        [0.7020411 , 0.7310075 , 0.00217595, ..., 0.74274397, 0.85443044,
         0.469761  ],
        [0.7890441 , 0.29349124, 0.4332237 , ..., 0.22531688, 0.13641207,
         0.955273  ],
        ...,
        [0.08417109, 0.47016835, 0.9377146 , ..., 0.82218635, 0.58684355,
      

Slicing is very fast, even if there are 100.000 samples.

In [19]:
# axis 0
%timeit X_on_disk[0]

11.5 µs ± 956 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [20]:
# axis 1
%timeit X_on_disk[..., 0]

16.2 µs ± 3.87 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [21]:
# axis 2
%timeit X_on_disk[:, 0]

14.6 µs ± 3.78 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [22]:
# aixs 0,1
%timeit X_on_disk[0, 0]

13.5 µs ± 983 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


To compare how fast you can slice a np.memmap, let's create a smaller array that I can fit in memory (X_in_memory). This is 10 times smaller (100 MB) than the one on disk.

In [23]:
X_in_memory_small = np.random.rand(10000, 50, 512)

In [24]:
%timeit X_in_memory_small[0]

661 ns ± 28.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Let's create the same array on disk. It's super simple:

In [36]:
np.save('./data/X_on_disk_small.npy', X_in_memory_small)
X_on_disk_small = np.load('./data/X_on_disk_small.npy', mmap_mode='r')

In [26]:
%timeit X_on_disk_small[0]

11.7 µs ± 515 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


This is approx. 17 slower than having arrays on disk, although it's still pretty fast.

However, if we use the itemified version, it's much faster:

In [27]:
%timeit X_on_disk_as_items[0]

2.39 µs ± 90.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


This is much better! So now you can access 1 of multiple items on disk with a pretty good performance.

#### Calculating stats: mean and std

Another benefit of using arrays on disk is that you can calculate the mean and std deviation of the entire dataset. 

It takes a considerable time since the array is very big (10GB), but it's feasible:

- mean (0.4999966):  1 min 45 s
- std  (0.2886839): 11 min 43 s 

in my laptop. 
If you need them, you could calculate these stats once, and store the results (similar to ImageNet stats).
However, you usually need to claculate these metrics for labeled (train) datasets, that tend to be smaller.

In [85]:
# X_mean = X_on_disk.mean()
# X_mean

0.4999966

In [86]:
# X_std = X_on_disk.std()
# X_std

0.28868386

#### Conversion into a tensor

Conversion from an array on disk slice into a tensor is also very fast:

In [28]:
torch.from_numpy(X_on_disk[0])

tensor([[0.2627, 0.8109, 0.3246,  ..., 0.2384, 0.6776, 0.3310],
        [0.7845, 0.6812, 0.3842,  ..., 0.5384, 0.0115, 0.1324],
        [0.6679, 0.5671, 0.0231,  ..., 0.7735, 0.2046, 0.9898],
        ...,
        [0.3863, 0.5770, 0.1810,  ..., 0.9798, 0.5666, 0.6019],
        [0.0903, 0.0175, 0.5882,  ..., 0.2834, 0.6954, 0.9360],
        [0.9186, 0.0125, 0.4146,  ..., 0.8035, 0.9133, 0.5419]])

In [29]:
X_on_disk_small_0 = X_on_disk_small[0]
X_in_memory_small_0 = X_in_memory_small[0]

In [30]:
%timeit torch.from_numpy(X_on_disk_small_0)

9.54 µs ± 25.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [31]:
%timeit torch.from_numpy(X_in_memory_small_0 )

9.66 µs ± 418 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


So it takes the same time to convert from numpy.memmap or from a np.array in memory is the same.

#### Combined operations: slicing plus conversion to tensor

Let's now check performance of the combined process: slicing plus conversion to a tensor. Based on what we've seen there are 3 options: 

- slice np.array in memory + conversion to tensor
- slice np.memamap on disk + conversion to tensor
- slice itemified np.memmap + converion to tensor

In [32]:
%timeit torch.from_numpy(X_in_memory_small[0])

17 µs ± 4.3 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [33]:
%timeit torch.from_numpy(X_on_disk_small[0])

38.2 µs ± 6.91 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [34]:
X_on_disk_small_as_items = itemify(X_on_disk_small)

In [35]:
%timeit torch.from_numpy(X_on_disk_small_as_items[0][0])

19.6 µs ± 2.57 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


So this last method is **almost as fast as having the array in memory**!! This is an excellent outcome, since slicing arrays in memory is a highly optimized operation. 

And we have the benefit of having access to very large datasets if needed.

## Summary

We now have a very efficient way to work with very large numpy arrays.

The process is very simple:

- create and save the array on disk (as described before)
- load it with a mmap_mode='r'
- itemify the array/s

So my recommendation would be:

- use numpy arrays in memory when possible (if your data fits in memory)
- use numpy memmap (arrays on disk) when data doesn't fit. You will still have a great performance.